# Autoencoder Analysis for CNN Version
As noted in the workbook using the convolutional neural network version of the autoencoder, training takes a **long** time.   So this notebook will be used to analyze the output of the trained AE.   You can use this for both the short version you train in the jupyter notebook, as well as the version you train using straight python.

This method of saving data to files - especially for the longer jobs you might run - and then analyzing them in a jupyter notebook, can be helpful for some of the work you might do for your projects.

## Load the models and output pickle file
Note that we call the history object we load from pickle "history_history", not histroy.history.

In [ ]:
from keras.models import load_model
import pickle

short = True
if short:
    encoder = load_model('fully_trained_encoder_cnn_small.h5')
    decoder = load_model('fully_trained_decoder_cnn_small.h5')
    autoencoder = load_model('fully_trained_autoencoder_cnn_small.h5')
    history_history = pickle.load(open('history_cnn_small.pkl', 'rb')) 
else:
    encoder = load_model('fully_trained_encoder_cnn.h5')
    decoder = load_model('fully_trained_decoder_cnn.h5')
    autoencoder = load_model('fully_trained_autoencoder_cnn.h5')
    history = pickle.load(open('history_cnn.pkl', 'rb')) 
    history_history = history.history

print("history",history_history)

## Load the data
We will want to run on the data for some of the performance tests, particularly the test data.  Note that we don't load into a pandas datframe the **training** data: pandas dataframes can be **huge** and to save memory we won't load the training data (since we won't do anything woith it in this workbook).

In [ ]:
from keras.datasets import mnist
import numpy as np
import scipy.io as sio
#
# See this for more info: https://arxiv.org/pdf/1702.05373.pdf
mat = sio.loadmat('/fs/scratch/PAS1043/physics6820/emnist/matlab/emnist-byclass.mat')
#print(mat)

data = mat['dataset']

ex_train = data['train'][0,0]['images'][0,0]
ey_train = data['train'][0,0]['labels'][0,0]
ex_test = data['test'][0,0]['images'][0,0]
ey_test = data['test'][0,0]['labels'][0,0]

ex_train = ex_train.reshape( (ex_train.shape[0], 28,28), order='F')
ex_test = ex_test.reshape( (ex_test.shape[0], 28,28), order='F')

ex_train = ex_train.reshape( (ex_train.shape[0], 784))
ex_test = ex_test.reshape( (ex_test.shape[0], 784))
ex_train = ex_train.astype('float32') / 255.
ex_test = ex_test.astype('float32') / 255.


import pandas as pd

#df_train = pd.DataFrame(ex_train)
#df_train['label'] = ey_train
#df_digits_train = df_train[df_train['label']<=9]
#x_train = df_digits_train.iloc[:50000,:784].values
#x_train = x_train.reshape((x_train.shape[0],28,28,1))
#y_train = df_digits_train.loc[:50000,'label'].values

df_test = pd.DataFrame(ex_test)
df_test['label'] = ey_test
df_digits_test = df_test[df_test['label']<=9]
x_test = df_digits_test.iloc[:,:784].values
x_test = x_test.reshape((x_test.shape[0],28,28,1))
y_test = df_digits_test['label'].values

#
from keras.utils import to_categorical

#y_train_labels_cat = to_categorical(y_train)
y_test_labels_cat = to_categorical(y_test)


In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

# Get training and test loss histories
training_loss = history_history['loss']
test_loss = history_history['val_loss']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training Loss', 'Test Loss'])
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.show();

In [ ]:
# Get training and test loss histories
training_loss = history_history['mean_squared_error']
test_loss = history_history['val_mean_squared_error']

# Create count of the number of epochs
epoch_count = range(1, len(training_loss) + 1)

# Visualize loss history
plt.plot(epoch_count, training_loss, 'r--')
plt.plot(epoch_count, test_loss, 'b-')
plt.legend(['Training MSE', 'Test MSE'])
plt.xlabel('Epoch')
plt.ylabel('MSE')
plt.show();

## Performance, Part II
Next we can try the "eye test": do the images at output look close to the images at input?

To test this, we first run all of our test images through the **predict** function of our network:

In [ ]:
decoded_imgs = autoencoder.predict(x_test)
print("decoded_imgs.shape",decoded_imgs.shape)

In [ ]:
import matplotlib.pyplot as plt

n = 10  # how many digits we will display
plt.figure(figsize=(20, 4))
for i in range(n):
    # display original
    ax = plt.subplot(2, n, i + 1)
    plt.imshow(x_test[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)

    # display reconstruction
    ax = plt.subplot(2, n, i + 1 + n)
    plt.imshow(decoded_imgs[i].reshape(28, 28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

## Visualizing the filters of a CNN
One thing we have not yet focused on is exactly what our networks are learning.   Clearly the encoder is finding some structure in the digits, and the decoder can then take the structure and recreate at least somewhat the original digits.  

If we use the **summary** method of a keras model, we can inspect the structure a bit.  Let's do that for the encoder.

When we do this, we will see the output below.  Focusing on just the enocder convolutional layers we see that:
1.  The first convolutional layer has 16 filters (the last dimension in the Output Shape column)
2.  The second convolutional layer has 8 filters 
3.  The third convolutional layer has 8 filters 


In [ ]:
print(encoder.summary())


## Visualizing the filters of a CNN (continued)
To understand what these filters are doing, we will use a method from Chapter 5 of the Deep Learning with Python text.   The idea is to find an image (starting from a randomly initialized image) which causes each of the filters to respond maximally.   In some sense, this is the stucture that the filter is "looking for" in the input images.

Here is the relevant code from Chapter 5:

In [ ]:
import numpy as np
from keras import backend as K

def deprocess_image(x):
    x -= x.mean()
    x /= (x.std() + 1e-5)
#    x *= 0.1
    x *= 0.3
    
    x+= 0.5
    x = np.clip(x,0,1)
    
    x*= 255
    x = np.clip(x,0,255).astype('uint8')
    return x

def generate_pattern(model,layer_name,filter_index,size=28):
    layer_output = model.get_layer(layer_name).output
    loss = K.mean(layer_output[:,:,:,filter_index])
    grads = K.gradients(loss,model.input)[0]
    grads /= (K.sqrt(K.mean(K.square(grads))) + 1e-5)
    iterate = K.function([model.input],[loss,grads])
    input_img_data = np.random.random((1,size,size,1))*20+128.0

    step = 1.0
    for i in range(40):
        loss_value,grads_value = iterate([input_img_data])
        input_img_data += grads_value * step
    
    img = input_img_data[0]
    return deprocess_image(img)

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(20, 4))
for i in range(16):
    # display original
    ax = plt.subplot(2, n, i + 1)
    thisImg = generate_pattern(encoder,'conv2d_1',i)
    plt.imshow(thisImg.reshape(28,28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

plt.figure(figsize=(20, 4))
for i in range(8):
    # display original
    ax = plt.subplot(2, n, i + 1)
    thisImg = generate_pattern(encoder,'conv2d_2',i)
    plt.imshow(thisImg.reshape(28,28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()

plt.figure(figsize=(20, 4))
for i in range(8):
    # display original
    ax = plt.subplot(2, n, i + 1)
    thisImg = generate_pattern(encoder,'conv2d_3',i)
    plt.imshow(thisImg.reshape(28,28))
    plt.gray()
    ax.get_xaxis().set_visible(False)
    ax.get_yaxis().set_visible(False)
plt.show()
